In [1]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json
from pprint import pprint

# PII 엔터티에 대한 텍스트 분석

Presidio Analyzer를 사용하여 텍스트를 분석하여 PII 엔터티를 식별합니다.

Presidio 분석기는 미리 정의된 엔터티 인식기를 사용하고 있으며 사용자 지정 인식기를 만드는 옵션을 제공합니다.

다음 코드 샘플은 다음을 수행합니다.

1. 분석기 엔진 설정: NLP 모듈(기본적으로 spaCy 모델) 및 기타 PII 인식기 로드

2. "PHONE_NUMBER" 항목 유형에 대한 분석 결과를 얻으려면 분석기를 호출하세요.

In [2]:
text_to_anonymize = "His name is Mr. Jones and his phone number is 212-555-5555"

In [3]:
analyzer = AnalyzerEngine()
analyzer_results = analyzer.analyze(text=text_to_anonymize, entities=["PHONE_NUMBER"], language='en')

print(analyzer_results)

[type: PHONE_NUMBER, start: 46, end: 58, score: 0.75]


# 사용자 지정 PII 엔터티 인식기 만들기

Presidio Analyzer는 사전 정의된 엔티티 인식기 세트와 함께 제공됩니다.

또한 사용자 지정 인식기를 생성 하여 분석기 기본 코드를 변경하지 않고 새 인식기를 추가할 수 있습니다 .

PatternRecognizer다음 예 에서는 분석된 텍스트에서 제목과 대명사를 식별하기 위해 두 개의 새로운 유형의 인식기를 생성 합니다.

A PatternRecognizer는 정규식 또는 거부 목록을 사용하는 PII 엔터티 인식기입니다.

다음 코드 샘플은 다음을 수행합니다.

1. 사용자 지정 인식기 만들기

2. 분석기에 새 사용자 정의 인식기 추가

3. 새 인식기에서 결과를 얻으려면 분석기를 호출하세요.

In [4]:
titles_recognizer = PatternRecognizer(supported_entity="TITLE",
                                      deny_list=["Mr.","Mrs.","Miss"])

pronoun_recognizer = PatternRecognizer(supported_entity="PRONOUN",
                                       deny_list=["he", "He", "his", "His", "she", "She", "hers" "Hers"])

analyzer.registry.add_recognizer(titles_recognizer)
analyzer.registry.add_recognizer(pronoun_recognizer)

analyzer_results = analyzer.analyze(text=text_to_anonymize,
                            entities=["TITLE", "PRONOUN"],
                            language="en")
print(analyzer_results)


[type: PRONOUN, start: 0, end: 3, score: 1.0, type: TITLE, start: 12, end: 15, score: 1.0, type: PRONOUN, start: 26, end: 29, score: 1.0]


Presidio Analyzer를 호출하고 구성된 모든 인식기(기본 및 새 사용자 정의 인식기)로 분석 결과를 얻으십시오.

In [5]:
analyzer_results = analyzer.analyze(text=text_to_anonymize, language='en')

analyzer_results

[type: PRONOUN, start: 0, end: 3, score: 1.0,
 type: TITLE, start: 12, end: 15, score: 1.0,
 type: PRONOUN, start: 26, end: 29, score: 1.0,
 type: PERSON, start: 16, end: 21, score: 0.85,
 type: PHONE_NUMBER, start: 46, end: 58, score: 0.75]

# 식별된 PII 엔티티로 텍스트 익명화

Presidio Anonymizer는 Presidio Analyzer 결과를 반복하고 식별된 텍스트에 대한 익명화 기능을 제공합니다.

익명화는 대체, 수정, 마스크, 해시 및 암호화의 5가지 유형의 익명화 도구를 제공합니다. 기본값은 교체 입니다.


다음 코드 샘플은 다음을 수행합니다.

1. 익명화 엔진 설정

2. 익명화 요청 생성 - 익명화할 텍스트, 적용할 익명화 목록 및 분석기 요청 결과

3. 텍스트를 익명화

In [6]:
anonymizer = AnonymizerEngine()

anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize,
    analyzer_results=analyzer_results,    
    operators={"DEFAULT": OperatorConfig("replace", {"new_value": ""}), 
                        "PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True}),
                        "TITLE": OperatorConfig("redact", {})}
)

print(f"text: {anonymized_results.text}")
print("detailed response:")

pprint(json.loads(anonymized_results.to_json()))

text: <PRONOUN> name is  <PERSON> and <PRONOUN> phone number is ************
detailed response:
{'items': [{'end': 70,
            'entity_type': 'PHONE_NUMBER',
            'operator': 'mask',
            'start': 58,
            'text': '************'},
           {'end': 41,
            'entity_type': 'PRONOUN',
            'operator': 'replace',
            'start': 32,
            'text': '<PRONOUN>'},
           {'end': 27,
            'entity_type': 'PERSON',
            'operator': 'replace',
            'start': 19,
            'text': '<PERSON>'},
           {'end': 18,
            'entity_type': 'TITLE',
            'operator': 'redact',
            'start': 18,
            'text': ''},
           {'end': 9,
            'entity_type': 'PRONOUN',
            'operator': 'replace',
            'start': 0,
            'text': '<PRONOUN>'}],
 'text': '<PRONOUN> name is  <PERSON> and <PRONOUN> phone number is '
         '************'}
